In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import open3d as o3d
import copy

from utils import readXML, Intrinsics, Extrinsics, RGBD, demo_crop_geometry, draw_registration_result, pick_points, demo_manual_registration

In [3]:
xml_dir = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/intrinsics_extrinsics/"

intrinsics = Intrinsics(xml_dir)
extrinsics = Extrinsics(xml_dir)
rgbds = RGBD(xml_dir, recording="20210709_004128")

# Stitching point clouds

### camera 020122061233

In [4]:
camera = "020122061233"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_1 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic)

#o3d.visualization.draw_geometries([pcd_1])

### camera 821312060044

In [5]:
camera = "821312060044"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_2 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, np.linalg.inv(extrinsic))


### camera 020122061651

In [6]:
camera = "020122061651"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_3 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, np.linalg.inv(extrinsic))


### camera 821312062243

In [7]:
camera = "821312062243"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_4 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, np.linalg.inv(extrinsic))


### Stitched Point Cloud

In [8]:
pcds = pcd_1 + pcd_2 + pcd_3 + pcd_4

o3d.visualization.draw_geometries([pcds])

# ICP


obj_pcd = o3d.io.read_point_cloud(obj_path)
obj_mesh = o3d.io.read_triangle_mesh(obj_path)

translation = obj_pcd.get_center()
obj_pcd.translate(translation)
obj_mesh.translate(translation)

obj_pcd.scale(0.001, center=(0,0,0))
obj_mesh.scale(0.001, center=(0,0,0))

o3d.visualization.draw_geometries([obj_pcd, obj_mesh])

In [30]:
obj_name = "005_tomato_soup_can"
obj_name = "003_cracker_box"
obj_name = "005_tomato_soup_can"

obj_path = f"C:/Users/lucas/Desktop/UPC/MIT/tactile2object/estimate_object_pose/ycb_converted/{obj_name}.ply"

obj_pcd = o3d.io.read_point_cloud(obj_path)
obj_mesh = o3d.io.read_triangle_mesh(obj_path)

obj_pcd.scale(0.001, center=(0,0,0))
obj_mesh.scale(0.001, center=(0,0,0))

TriangleMesh with 8178 points and 16384 triangles.

In [31]:
source = copy.deepcopy(obj_pcd)
target = copy.deepcopy(pcds)
transformation = demo_manual_registration(source, target)

Demo for manual ICP
Visualization of two point clouds before manual alignment

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] No point has been picked.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D INFO] Picked point #8163 (0.022, 0.095, 0.1) to add in queue.
[Open3D INFO] Picked point #4036 (-0.041, 0.074, 0.1) to add in queue.
[Open3D INFO] Picked point #755 (-0.041, 0.076, 0.0011) to add in queue.


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #42049 (0.053, -0.15, 0.53) to add in queue.
[Open3D INFO] Picked point #53273 (-0.0076, -0.13, 0.52) to add in queue.
[Open3D INFO] Picked point #680539 (-0.00035, -0.068, 0.6) to add i

In [32]:
transformation

array([[ 0.99736015, -0.03803079, -0.06185787,  0.04239608],
       [-0.06796566, -0.78880544, -0.61087368, -0.01322389],
       [-0.02556181,  0.61346527, -0.7893079 ,  0.55232266],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [33]:
obj_mesh_t = copy.deepcopy(obj_mesh).transform(transformation)
obj_005_t = obj_mesh_t

In [35]:
obj_003_t.compute_vertex_normals()
obj_003_t.paint_uniform_color([1, 0.706, 0])

obj_005_t.compute_vertex_normals()
obj_005_t.paint_uniform_color([1, 0.706, 0])

o3d.visualization.draw_geometries([pcds, obj_003_t, obj_005_t])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
